Delete all variables in the current environment (if you have already run some cells) - clean state

In [1]:
%reset

Import all necessary packages

In [2]:
import pandas as pd
import os
import networkx as nx
from matplotlib import pyplot as plt
import shutil

Replace with the path to the root folder of the project

In [3]:
rootdir_path = '/home/andreistoica12/research-internship'

Replace with the path to the folder where we store the dataset

In [4]:
data_path = '/home/andreistoica12/research-internship/data/covaxxy-csv'

Create 2 subfolders to store important files and graphs, respectively. If they already existed (from previous runnings of the project), delete the folders and their contents and create empty folders to store the current files and graphs, relevant to the current state of the project.

In [5]:
files_path = os.path.join(rootdir_path, 'files')
if os.path.exists(files_path):
   shutil.rmtree(files_path, ignore_errors=False, onerror=None)
os.makedirs(files_path)

graphs_path = os.path.join(rootdir_path, 'graphs')
if os.path.exists(graphs_path):
   shutil.rmtree(graphs_path, ignore_errors=False, onerror=None)
os.makedirs(graphs_path)

In [6]:
os.listdir(data_path)

['tweet_ids--2021-03-02.csv',
 'tweet_ids--2021-03-03.csv',
 'tweet_ids--2021-03-05.csv',
 'tweet_ids--2021-03-04.csv',
 'tweet_ids--2021-03-01.csv']

In [9]:
first_batch_path = os.path.join(data_path, os.listdir(data_path)[0])

In [11]:
first_batch = pd.read_csv(first_batch_path)

In [12]:
first_batch

,Unnamed: 0,created_at,tweet_id,author_id,text,followers_count,following_count
0,0,2021-03-01T23:59:55.000Z,1366538722066395138,201963182,RT @briantylercohen: So Trump got the vaccine ...,865,2126
1,1,2021-03-01T23:59:55.000Z,1366538722955563008,1301587100509241344,RT @Cal_OES: Protecting Our Communities from #...,0,4
2,2,2021-03-01T23:59:55.000Z,1366538724092170246,330378192,RT @drmt: @StephenKing Congrats @StephenKing !...,20884,711
3,3,2021-03-01T23:59:55.000Z,1366538723819741192,198019557,The fucking gall of these cretins. They're con...,121,520
4,4,2021-03-01T23:59:55.000Z,1366538724255948802,2790227032,RT @briantylercohen: So Trump got the vaccine ...,1592,5009
...,...,...,...,...,...,...,...
690235,690235,2021-03-02T03:03:09.000Z,1366584834034987017,90485501,My son Gavi is raising money for Chai Lifeline...,146,588
690236,690236,2021-03-02T03:03:09.000Z,1366584835284860929,2453661493,RT @Matthew_4_Trump: The people who are agains...,3419,5094
690237,690237,2021-03-02T03:03:09.000Z,1366584836010475520,15208773,@FlyinSparxx @MollyJongFast agree. And I belie...,2375,5001
690238,690238,2021-03-02T03:03:09.000Z,1366584836404649991,510244786,RT @rishibagree: 3 News that turn out to be #F...,529,235


In [13]:
type(first_batch['author_id'])

pandas.core.series.Series

In [14]:
first_batch['author_id'].size

690240

In [17]:
first = first_batch['author_id']

In [19]:
first.drop_duplicates(inplace=True)

In [20]:
first.size

397428